In [1]:
import pandas as pd
import geojson
import ast
# from bokeh.models.sources import ColumnDataSource

    cd Desktop
    activate nyc-buildings
    jupyter notebook

In [25]:
mepl = pd.read_pickle('most_enumerate_private_landowners.p')
wpl = pd.read_pickle('wealthiest_private_landowners.p')
lpl = pd.read_pickle('largest_private_landowners.p')

Due to several errors that I didn't catch the first time around I need to apply a few hotfixes.

In [26]:
usbna_places = mepl['Places'][5]
nyu_places = mepl['Places'][6]

In [27]:
# Ignore the warnings, as always.
mepl['Places'][5] = nyu_places
mepl['Places'][6] = usbna_places

/Users/Honors/anaconda/envs/nyc-buildings/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/Honors/anaconda/envs/nyc-buildings/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [28]:
wpl.iloc[10, 2] = "{'200 PARK AVENUE': [40.7535, 73.9768]}"

In [29]:
lpl['Places'][8] = "{'1230 AVENUE OF THE AMERICAS': [40.758883, -73.980372], '1260 AVENUE OF THE AMERICAS': [40.75995, -73.979728]}"

/Users/Honors/anaconda/envs/nyc-buildings/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [30]:
wpl['Places'][20] = "{'180 FT WASHINGTON AVENUE': [40.840707, -73.9413], '161 FT WASHINGTON AVENUE': [40.8413, -73.9435]}"

/Users/Honors/anaconda/envs/nyc-buildings/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [31]:
wpl['Places'][19] = "{'200 PARK AVENUE': [40.7638, -73.9746]}"

/Users/Honors/anaconda/envs/nyc-buildings/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [32]:
wpl.iloc[5, 0] = "New York Convention Center Corporation"

In [45]:
wpl['Places'][2] = str({'1230 AVENUE OF THE AMERICAS': [40.75997, -73.9799]})

/Users/Honors/anaconda/envs/nyc-buildings/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [47]:
# for l in [zip(mepl['Owner'], mepl['Places']), zip(lpl['Owner'], lpl['Places']), zip(wpl['Owner'], wpl['Places'])]:
for (entity, location_set, address_count) in zip(mepl['Owner'], mepl['Places'], mepl['Addresses']):
    features = []
    for address, (latitude, longitude) in ast.literal_eval(location_set).items():
        features.append(geojson.Feature(geometry=geojson.Point((longitude, latitude)), properties={'address': address}))
        # print(entity, address, latitude, longitude)
    with open('geojson/mepl/' + entity + '.geojson', 'w') as file:
        file.write(geojson.dumps(geojson.FeatureCollection(features, properties={'addresses': str(address_count)}), sort_keys=True))
        
for (entity, location_set, square_footage) in zip(lpl['Owner'], lpl['Places'], lpl['Sq. Footage']):
    features = []
    for address, (latitude, longitude) in ast.literal_eval(location_set).items():
        features.append(geojson.Feature(geometry=geojson.Point((longitude, latitude)), properties={'address': address}))
        # print(entity, address, latitude, longitude)
    with open('geojson/lpl/' + entity + '.geojson', 'w') as file:
        file.write(geojson.dumps(geojson.FeatureCollection(features, properties={'square footage': "{:,.2f}".format(int(square_footage))[:-3]}), sort_keys=True))
        
for (entity, location_set, total_value) in zip(wpl['Owner'], wpl['Places'], wpl['Total Value']):
    features = []
    for address, (latitude, longitude) in ast.literal_eval(location_set).items():
        features.append(geojson.Feature(geometry=geojson.Point((longitude, latitude)), properties={'address': address}))
        # print(entity, address, latitude, longitude)
    with open('geojson/wpl/' + entity + '.geojson', 'w') as file:
        file.write(geojson.dumps(geojson.FeatureCollection(features, properties={'total value': "{:,.2f}".format(int(total_value))[:-3]}), sort_keys=True))

In [11]:
%ls

both-tabs-and-commas-screencap.png      most_enumerate_private_landowners.p
data-munging.ipynb                      nyc.json
data-scribbles.ipynb                    nyc_building_sales.csv
data-visualization.ipynb                road.html
final-data-visualization.ipynb          rolling-sales-data-excel-screencap.png
final-visualization.html                test.html
geojson/                                wealthiest_private_landowners.p
largest_private_landowners.p


According to the dataset the Rockefeller Plaza holding company is in Brooklyn...

In [12]:
# currently shows Columbia's property, as well a useless radio group buttion

from bokeh.io import output_file, show, vform, vplot, gridplot, hplot, curdoc, output_notebook
from bokeh.models import GMapPlot, GMapOptions, DataRange1d, ColumnDataSource, Circle, PanTool, WheelZoomTool, HoverTool, HBox, VBox, VBoxForm, BoxZoomTool
from bokeh.models.widgets import RadioButtonGroup, Slider, Select

addresses = literal_eval(wpl['Places'][1]) # for now just plot Columbia
map_options = GMapOptions(lat=40.6773, lng=-73.9857, map_type = "terrain", zoom=10)
# TODO: plot focus needs to be adjusted further to fix Bronx mapping.

GMap = GMapPlot(x_range = DataRange1d(),
                y_range = DataRange1d(),
                map_options = map_options ,
                title = "Eneumerate",
                plot_width=960,
                plot_height=600)

lats = []
longs = []
names = []
for address in addresses.keys():
    names.append(address)
    lats.append(addresses[address][0])
    longs.append(addresses[address][1])

Property = ColumnDataSource(data=dict(lat = lats, 
                                      lon = longs,
                                      name = names))

Address_Circle = Circle(x="lon", y="lat", size = 10)

GMap.add_glyph( Property, Address_Circle)
hover = HoverTool( tooltips=[(" ", "@name")])
GMap.add_tools(hover, PanTool(), WheelZoomTool(), BoxZoomTool())
GMap.logo = None
# GMap.toolbar_location = None

radio_button_group = RadioButtonGroup(labels=["Enumerate", "Value", "Square Footage"], active=0)

select = Select(options=list(mepl['Owner']))

def update(attrname, old, new):
    if radio_button_group.active == 0:
        df = mepl
    elif radio_button_group.active == 1:
        df = lpl
    else:
        df = wpl
    addresses = literal_eval(df['Places'][1])
    names = lats = longs = []
    for address in addresses.keys():
        names.append(address)
        lats.append(addresses[address][0])
        longs.append(addresses[address][1])
    Property.data = dict(
        lat = lats,
        lon = longs,
        name = names
    )

# for r, s in [radio_button_group, select]:
    
plot = vform(GMap, radio_button_group, select)

for w in [radio_button_group, select]:
    w.on_change('value', update)

curdoc().add_root(HBox(children=[radio_button_group, select, plot], width=800))

output_file("test.html")
# output_notebook()
show(plot)

NameError: name 'literal_eval' is not defined

In [ ]:
from bokeh.models import TextInput, Select
from bokeh.plotting import Figure

In [ ]:
# # Create Input controls
# reviews = Slider(title="Minimum number of reviews", value=80, start=10, end=300, step=10)
# min_year = Slider(title="Year released", start=1940, end=2014, value=1970, step=1)
# max_year = Slider(title="End Year released", start=1940, end=2014, value=2014, step=1)
# oscars = Slider(title="Minimum number of Oscar wins", start=0, end=4, value=0, step=1)
# boxoffice = Slider(title="Dollars at Box Office (millions)", start=0, end=800, value=0, step=1)
# # genre = Select(title="Genre", value="All",
# #                options=open(join(dirname(__file__), 'genres.txt')).read().split())
# director = TextInput(title="Director name contains")
# cast = TextInput(title="Cast names contains")
# # x_axis = Select(title="X Axis", options=sorted(axis_map.keys()), value="Tomato Meter")
# # y_axis = Select(title="Y Axis", options=sorted(axis_map.keys()), value="Number of Reviews")

# # Create Column Data Source that will be used by the plot
# source = ColumnDataSource(data=dict(x=[], y=[], color=[], title=[], year=[], revenue=[]))

# hover = HoverTool(tooltips=[
#     ("Title","@title"),
#     ("Year", "@year"),
#     ("$", "@revenue")
# ])

# p = Figure(plot_height=600, plot_width=800, title="", toolbar_location=None, tools=[hover])
# p.circle(x="x", y="y", source=source, size=7, color="color", line_color=None, fill_alpha="alpha")

# # def select_movies():
# #     genre_val = genre.value
# #     director_val = director.value.strip()
# #     cast_val = cast.value.strip()
# #     selected = movies[
# #         (movies.Reviews >= reviews.value) &
# #         (movies.BoxOffice >= (boxoffice.value * 1e6)) &
# #         (movies.Year >= min_year.value) &
# #         (movies.Year <= max_year.value) &
# #         (movies.Oscars >= oscars.value)
# #     ]
# #     if (genre_val != "All"):
# #         selected = selected[selected.Genre.str.contains(genre_val)==True]
# #     if (director_val != ""):
# #         selected = selected[selected.Director.str.contains(director_val)==True]
# #     if (cast_val != ""):
# #         selected = selected[selected.Cast.str.contains(cast_val)==True]
# #     return selected

# def update(attrname, old, new):
#     df = select_movies()
#     x_name = axis_map[x_axis.value]
#     y_name = axis_map[y_axis.value]

#     p.xaxis.axis_label = x_axis.value
#     p.yaxis.axis_label = y_axis.value
#     p.title = "%d movies selected" % len(df)
#     source.data = dict(
#         x=df[x_name],
#         y=df[y_name],
#         color=df["color"],
#         title=df["Title"],
#         year=df["Year"],
#         revenue=df["revenue"],
#         alpha=df["alpha"],
#     )

# controls = [reviews, boxoffice, min_year, max_year, oscars, director, cast]
# # for control in controls:
# #     control.on_change('value', update)

# inputs = HBox(VBoxForm(*controls), width=300)

# # update(None, None, None) # initial load of the data

# curdoc().add_root(HBox(inputs, p, width=1100))

# output_file('test.html')

# show(p)

In [ ]:
# ''' Present an interactive function explorer with slider widgets.
# Scrub the sliders to change the properties of the ``sin`` curve, or
# type into the title text box to update the title of the plot.
# Use the ``bokeh serve`` command to run the example by executing:
#     bokeh serve sliders.py
# at your command prompt. Then navigate to the URL
#     http://localhost:5006/sliders
# in your browser.
# '''
# import numpy as np

# from bokeh.plotting import Figure
# from bokeh.models import ColumnDataSource, HBox, VBoxForm
# from bokeh.models.widgets import Slider, TextInput
# from bokeh.io import curdoc

# # Set up data
# N = 200
# x = np.linspace(0, 4*np.pi, N)
# y = np.sin(x)
# source = ColumnDataSource(data=dict(x=x, y=y))


# # Set up plot
# plot = Figure(plot_height=400, plot_width=400, title="my sine wave",
#               tools="crosshair,pan,reset,resize,save,wheel_zoom",
#               x_range=[0, 4*np.pi], y_range=[-2.5, 2.5])

# plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)


# # Set up widgets
# text = TextInput(title="title", value='my sine wave')
# offset = Slider(title="offset", value=0.0, start=-5.0, end=5.0, step=0.1)
# amplitude = Slider(title="amplitude", value=1.0, start=-5.0, end=5.0)
# phase = Slider(title="phase", value=0.0, start=0.0, end=2*np.pi)
# freq = Slider(title="frequency", value=1.0, start=0.1, end=5.1)


# # Set up callbacks
# def update_title(attrname, old, new):
#     plot.title = text.value

# text.on_change('value', update_title)

# def update_data(attrname, old, new):

#     # Get the current slider values
#     a = amplitude.value
#     b = offset.value
#     w = phase.value
#     k = freq.value

#     # Generate the new curve
#     x = np.linspace(0, 4*np.pi, N)
#     y = a*np.sin(k*x + w) + b

#     source.data = dict(x=x, y=y)

# for w in [offset, amplitude, phase, freq]:
#     w.on_change('value', update_data)


# # Set up layouts and add to document
# inputs = VBoxForm(children=[text, offset, amplitude, phase, freq])
# layout = HBox(children=[inputs, plot], width=800)


# curdoc().add_root(layout)

# output_file('test2.html')
# show(layout)